<a href="https://colab.research.google.com/github/isabernardes/Homework/blob/master/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [9]:
!pip install sentinelsat

In [10]:
import os
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from pprint import pprint
from urllib.request import urlopen
import json
from shapely.geometry import Polygon
import shapely
import geojson as gj

from osgeo import ogr, osr, gdal
print(gdal.__version__)

import matplotlib.pyplot as plt
%matplotlib inline


2.2.3


In [11]:
api = SentinelAPI('isabernardes', 'Bananamor90', 'https://scihub.copernicus.eu/dhus')
longitude = 13.3901966
latitude = 52.5228906

products = api.query(f'POINT({longitude} {latitude})',
                     date = (date(2019, 6, 1), date(2020, 2, 29)),
                     platformname = 'Sentinel-2',
                     cloudcoverpercentage = (0, 30))

# convert to Pandas DataFrame
products_df = api.to_dataframe(products)

# sort and limit to first 5 sorted products
products_df_sorted = products_df.sort_values(['cloudcoverpercentage', 'ingestiondate'], ascending=[True, True])
products_df_sorted = products_df_sorted.head(1)

print(products_df_sorted)

Querying products: 100%|██████████| 119/119 [00:00<00:00, 298.05 products/s]

                                                                                  title  ... hv_order_tileid
3b19578c-412b-45c7-854c-ea89a9599ac7  S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_2...  ...           UU33U

[1 rows x 40 columns]


In [12]:
download_dir = '/content/drive/My Drive/work/IP4RS_lab/data'

api.download_all(products_df_sorted.index, directory_path=download_dir)

Downloading: 100%|██████████| 825M/825M [00:29<00:00, 27.9MB/s]
MD5 checksumming: 100%|██████████| 825M/825M [00:02<00:00, 342MB/s]


(OrderedDict([('3b19578c-412b-45c7-854c-ea89a9599ac7',
               {'Creation Date': datetime.datetime(2019, 7, 26, 16, 24, 11, 283000),
                'Ingestion Date': datetime.datetime(2019, 7, 26, 16, 23, 20, 812000),
                'Online': True,
                'date': datetime.datetime(2019, 7, 26, 10, 20, 31, 24000),
                'downloaded_bytes': 824870777,
                'footprint': 'POLYGON((12.004760925069467 53.21197782662703,13.648341514492438 53.24196354764364,13.678544065009483 52.25515074878083,12.071595116334858 52.22621087043558,12.004760925069467 53.21197782662703))',
                'id': '3b19578c-412b-45c7-854c-ea89a9599ac7',
                'md5': 'DF709FBC213AE9AFD7CE695DC5A98D2C',
                'path': '/content/drive/My Drive/work/IP4RS_lab/data/S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_20190726T121947.zip',
                'size': 824870777,
                'title': 'S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_20190726T121947',
           

In [13]:
!unzip /content/drive/My\ Drive/work/IP4RS_lab/data/S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_20190726T121947.zip -d /content/drive/My\ Drive/work/IP4RS_lab/data/ 

Archive:  /content/drive/My Drive/work/IP4RS_lab/data/S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_20190726T121947.zip
   creating: /content/drive/My Drive/work/IP4RS_lab/data/S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_20190726T121947.SAFE/
 extracting: /content/drive/My Drive/work/IP4RS_lab/data/S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_20190726T121947.SAFE/INSPIRE.xml  
   creating: /content/drive/My Drive/work/IP4RS_lab/data/S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_20190726T121947.SAFE/rep_info/
 extracting: /content/drive/My Drive/work/IP4RS_lab/data/S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_20190726T121947.SAFE/rep_info/S2_User_Product_Level-1C_Metadata.xsd  
   creating: /content/drive/My Drive/work/IP4RS_lab/data/S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_20190726T121947.SAFE/HTML/
 extracting: /content/drive/My Drive/work/IP4RS_lab/data/S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_20190726T121947.SAFE/HTML/star_bg.jpg  
 extracting: /content/drive/My Drive/work/IP

In [14]:
sen2_xml = '/content/drive/My Drive/work/IP4RS_lab/data/S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_20190726T121947.SAFE/MTD_MSIL1C.xml'
# open dataset
raster = gdal.Open(sen2_xml)
# print sentinel-2 information
raster_info = gdal.Info(sen2_xml)
print(raster_info)
# get subset from raster
datasets = raster.GetSubDatasets()
print(list(datasets))

bands_10m = list(datasets)[0]
bands_20m = list(datasets)[1]
bands_60m = list(datasets)[2]

Driver: SENTINEL2/Sentinel 2
Files: /content/drive/My Drive/work/IP4RS_lab/data/S2A_MSIL1C_20190726T102031_N0208_R065_T33UUU_20190726T121947.SAFE/MTD_MSIL1C.xml
Size is 512, 512
Coordinate System is `'
Metadata:
  CLOUD_COVERAGE_ASSESSMENT=0.0
  DATATAKE_1_DATATAKE_SENSING_START=2019-07-26T10:20:31.024Z
  DATATAKE_1_DATATAKE_TYPE=INS-NOBS
  DATATAKE_1_ID=GS2A_20190726T102031_021369_N02.08
  DATATAKE_1_SENSING_ORBIT_DIRECTION=DESCENDING
  DATATAKE_1_SENSING_ORBIT_NUMBER=65
  DATATAKE_1_SPACECRAFT_NAME=Sentinel-2A
  DEGRADED_ANC_DATA_PERCENTAGE=0.0
  DEGRADED_MSI_DATA_PERCENTAGE=0
  FOOTPRINT=POLYGON((12.004760925069467 53.21197782662703, 13.648341514492438 53.24196354764364, 13.678544065009483 52.25515074878083, 12.071595116334858 52.22621087043558, 12.004760925069467 53.21197782662703))
  GENERATION_TIME=2019-07-26T12:19:47.000000Z
  PREVIEW_GEO_INFO=Not applicable
  PREVIEW_IMAGE_URL=Not applicable
  PROCESSING_BASELINE=02.08
  PROCESSING_LEVEL=Level-1C
  PRODUCT_START_TIME=2019-07-26

In [15]:
ds10 = gdal.Open(bands_10m[0])

for i in range(ds10.RasterCount):
  print(ds10.GetRasterBand(i+1).GetDescription())

# ds10_info = gdal.Info(bands_10m[0])
# print(ds10_info)

ds20 = gdal.Open(bands_20m[0])

for i in range(ds20.RasterCount):
  print(ds20.GetRasterBand(i+1).GetDescription())

# ds20_info = gdal.Info(bands_20m[0])
# print(ds20_info)

ds60 = gdal.Open(bands_60m[0])

for i in range(ds60.RasterCount):
  print(ds60.GetRasterBand(i+1).GetDescription())

# ds60_info = gdal.Info(bands_60m[0])
# print(ds60_info)

# determine the coordinate and buffer information
tmxmin = 0
tmxmax = ds10.RasterXSize - 1
tmymin = 0
tmymax = ds10.RasterYSize - 1


data10 = np.rollaxis(
            ds10.ReadAsArray(xoff=tmxmin, yoff=tmymin, xsize=tmxmax - tmxmin + 1, ysize=tmymax - tmymin + 1, buf_xsize=tmxmax - tmxmin + 1,
                             buf_ysize=tmymax - tmymin + 1), 0, 3)

data20 = np.rollaxis(
            ds20.ReadAsArray(xoff=tmxmin // 2, yoff=tmymin // 2, xsize=(tmxmax - tmxmin + 1) // 2, ysize=(tmymax - tmymin + 1) // 2,
                             buf_xsize=(tmxmax - tmxmin + 1) // 2, buf_ysize=(tmymax - tmymin + 1) // 2), 0, 3)
# did not select B10, sinc it does not contain surface information
data60 = np.rollaxis(
          ds60.ReadAsArray(xoff=tmxmin // 6, yoff=tmymin // 6, xsize=(tmxmax - tmxmin + 1) // 6, ysize=(tmymax - tmymin + 1) // 6,
                             buf_xsize=(tmxmax - tmxmin + 1) // 6, buf_ysize=(tmymax - tmymin + 1) // 6), 0, 3)[:,:,[0,1]]

B4, central wavelength 665 nm
B3, central wavelength 560 nm
B2, central wavelength 490 nm
B8, central wavelength 842 nm
B5, central wavelength 705 nm
B6, central wavelength 740 nm
B7, central wavelength 783 nm
B8A, central wavelength 865 nm
B11, central wavelength 1610 nm
B12, central wavelength 2190 nm
B1, central wavelength 443 nm
B9, central wavelength 945 nm
B10, central wavelength 1375 nm


In [0]:
def visual_band(data, percentile_data=None):
  """
  clip image values and create uint8 image
  """
  if percentile_data is None:
      percentile_data = data
  mi, ma = np.percentile(percentile_data, (1, 99))
  band_data = np.clip(data, mi, ma)
  band_data = (band_data - mi) / (ma - mi)

  return (255*band_data).astype(np.uint8)